In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

In [18]:
import tensorflow as tf

# Force TensorFlow to use GPU 0
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Allow memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("GPU is set and memory growth is enabled.")
    except RuntimeError as e:
        print(e)

GPU is set and memory growth is enabled.


In [19]:
# Optional: check GPU
print("Available GPU:", tf.config.list_physical_devices('GPU'))

Available GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [20]:
# Set dataset paths
train_dir = '/kaggle/input/fer2013/train'
val_dir = '/kaggle/input/fer2013/test'

In [21]:
# Image generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.4,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    brightness_range=[0.6, 1.4]
)

val_datagen = ImageDataGenerator(rescale=1./255)

train = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

val = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [22]:
# Define CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotion classes
])

In [14]:
print(type(model))

<class 'keras.src.models.sequential.Sequential'>


In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save best model
checkpoint = ModelCheckpoint("b2_model.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
import os
num_workers = min(4, os.cpu_count() or 1)
# Train model
model.fit(train, validation_data=val, epochs=30, callbacks=[checkpoint])


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


I0000 00:00:1747413804.614898     112 service.cc:148] XLA service 0x7d6e14006e10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747413804.615648     112 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1747413804.945188     112 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/449 ━━━━━━━━━━━━━━━━━━━━ 37:15 5s/step - accuracy: 0.2344 - loss: 1.9393

I0000 00:00:1747413807.231654     112 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step - accuracy: 0.2343 - loss: 1.8385
Epoch 1: val_accuracy improved from -inf to 0.25146, saving model to b2_model.h5
449/449 ━━━━━━━━━━━━━━━━━━━━ 176s 382ms/step - accuracy: 0.2343 - loss: 1.8384 - val_accuracy: 0.2515 - val_loss: 1.8013
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.2550 - loss: 1.7988
Epoch 2: val_accuracy improved from 0.25146 to 0.26470, saving model to b2_model.h5
449/449 ━━━━━━━━━━━━━━━━━━━━ 47s 106ms/step - accuracy: 0.2550 - loss: 1.7987 - val_accuracy: 0.2647 - val_loss: 1.7525
Epoch 3/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.2620 - loss: 1.7777
Epoch 3: val_accuracy improved from 0.26470 to 0.29785, saving model to b2_model.h5
449/449 ━━━━━━━━━━━━━━━━━━━━ 47s 106ms/step - accuracy: 0.2620 - loss: 1.7777 - val_accuracy: 0.2979 - val_loss: 1.7099
Epoch 4/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.2680 - loss: 1.7647
Epoch 4: val_accuracy improved from 0.29785 to 0.304